# Predictor trained on experimental DMS data to distinguish damaging from neutral mutations

## Setup

* Data: exhausive first-principle evolutionary statistical energy calculation from EVmutation on ~ 10k proteins
* Feature vector: each variant is a **58-element vector** - 
  + **WT Amino acid** (one-hot encoding) [*20 features*]
  + **MUT Amino Acid** (one-hot encoding) [*20 features*]
  + **trinucleotide context**
    * position -1 (one-hot encoding) [*4 features*]
    * position 0 (6 possible substitutions, one-hot encoding) [*6 features*]
    * position +1 (one-hot encoding) [*4 features*]
  + phyloP **conservation score** [separately for positions -1, 0, 1] [*3 features*]
  + **Q(SASA)** [1 feature]
  
## Data source

* [EVmutation data portal](https://marks.hms.harvard.edu/evmutation/human_proteins.html) `effect.tar.gz` downloaded on 24 Sept 2021.

In [1]:
%load_ext rpy2.ipython

In [2]:
import numpy as np
import pandas as pd
import re
import pickle

In [3]:
# read in data
EVmutation = pd.read_csv('human_protein_predictions/EVmutation_human_protein_predictions.txt', sep=";")


In [4]:
EVmutation

,uniprot,pfam,mutant,pos,wt,subs,prediction_epistatic,prediction_independent,frequency,column_conservation
0,A0AV02,PF00324,L38A,38,L,A,-6.7742,-3.5671,0.002260,0.4888
1,A0AV02,PF00324,L38C,38,L,C,-7.7904,-6.6306,0.000000,0.4888
2,A0AV02,PF00324,L38D,38,L,D,-6.3475,-3.0068,0.003986,0.4888
3,A0AV02,PF00324,L38E,38,L,E,-6.4752,-3.4645,0.002509,0.4888
4,A0AV02,PF00324,L38F,38,L,F,-7.7904,-6.6306,0.000000,0.4888
...,...,...,...,...,...,...,...,...,...,...
32028175,Q9Y6Y8,PF00536,A704S,704,A,S,-3.0319,0.1986,0.040747,0.4645
32028176,Q9Y6Y8,PF00536,A704T,704,A,T,-4.2223,-0.5933,0.018462,0.4645
32028177,Q9Y6Y8,PF00536,A704V,704,A,V,-4.2239,-2.5868,0.002508,0.4645
32028178,Q9Y6Y8,PF00536,A704W,704,A,W,-7.2978,-6.7077,0.000000,0.4645


In [5]:
# set of proteins included
proteins = list(set(EVmutation['uniprot']))
print(len(proteins))
# include only those which are canonical uniprot entries
proteins = [p for p in proteins if p.find('-') == -1]
print(len(proteins))
print(proteins[:10])

6938
6671
['Q8IWL1', 'Q9Y6H3', 'Q9NRF2', 'Q6UW60', 'O00339', 'P20742', 'Q15796', 'Q15833', 'Q96GX5', 'O14832']


In [6]:
with open('human_protein_predictions/human_protein_included_list', 'w') as f:
    for p in proteins:
        f.write( p + '\n')

### Annotate features

* QSASA: Taken from ZoomVar
* MutSig, PhyloP scores: Use CDSMutSig and the phyloP bioconductor packages (adapted from Marija's code) in R

In [7]:
%%bash

# ZoomVar
head -1 $HOME/Documents/ZoomvarTCGA/protein_position_annotations_zoomvar_category_NEW.tsv > human_protein_predictions/ZoomVar.tsv
while read uniprot; do
    grep -E "^"$uniprot $HOME/Documents/ZoomvarTCGA/protein_position_annotations_zoomvar_category_NEW.tsv >> human_protein_predictions/ZoomVar.tsv
done < 'human_protein_predictions/human_protein_included_list'

In [8]:
%%R -i proteins
# Map genomic position, MutSig and phyloP scores

ensp <- ensembldb::proteins( EnsDb.Hsapiens.v86::EnsDb.Hsapiens.v86,
                     filter = AnnotationFilter::UniprotFilter(proteins),
                     columns = c("uniprot_id", "protein_id"))
# select only transcripts in MANE
MANE <- read.table(file = "../../MANE.GRCh38.v0.93.summary.txt.gz", sep = "\t", 
                   header = TRUE, comment.char = '', stringsAsFactors = FALSE, quote = '')
MANE$EnsProtID <- sapply(MANE$Ensembl_prot, function(x) unlist(strsplit(x, split = ".", fixed = TRUE))[1] )

Ensembl_MANE <- merge(ensp, MANE[, c(4, ncol(MANE))], by.x = "protein_id", by.y = "EnsProtID", all.x = TRUE)
Ensembl_MANE <- Ensembl_MANE[ which(!is.na(Ensembl_MANE[, "symbol"])), ]
print(head(Ensembl_MANE))
print(nrow(Ensembl_MANE))
                         
# Use CDSMutSig
library(CDSMutSig)
mutsigs <- lapply(Ensembl_MANE$protein_id, function(id){
  cat(paste0(id, " ...\n"))
  codons <- mapCodon(id, EnsDb.Hsapiens.v86::EnsDb.Hsapiens.v86,
                     BSgenome.Hsapiens.NCBI.GRCh38::Hsapiens, 
                     seqLevelStyle = "NCBI")
  if( is.null( codons ) ) return(NULL)
  codons_muts <- mapMut(codons, getPossibleMissenseSNVs())
  codons_mutsigs <- mapMutSig(id, nrow(codons), codons_muts, 
                              EnsDb.Hsapiens.v86::EnsDb.Hsapiens.v86,
                              BSgenome.Hsapiens.NCBI.GRCh38::Hsapiens, 
                              seqLevelStyle = "NCBI")
  codons_mutsigs$ensp <- id  
  codons_mutsigs
})
       
mutsigs <- do.call("rbind", mutsigs)
mutsigs <- merge( ensp, mutsigs, by.x = "protein_id", by.y = "ensp" )
write.table(mutsigs, 'human_protein_predictions/MutSig.tsv', sep = "\t",
            col.names = TRUE, row.names = FALSE, quote = FALSE)

       protein_id uniprot_id  symbol
1 ENSP00000001146     Q9NR63 CYP26B1
2 ENSP00000002165     Q9BTY2   FUCA2
3 ENSP00000002596     O14792  HS3ST1
4 ENSP00000002829     Q13275  SEMA3F
5 ENSP00000003084     P13569    CFTR
7 ENSP00000005178     Q16654    PDK4
[1] 5692
ENSP00000001146 ...


R[write to console]: Fetching CDS for 1 proteins ... 
R[write to console]: 1 found

R[write to console]: Checking CDS and protein sequence lengths ... 
R[write to console]: 1/1 OK

R[write to console]: Fetching CDS for 1 proteins ... 
R[write to console]: 1 found

R[write to console]: Checking CDS and protein sequence lengths ... 
R[write to console]: 1/1 OK



ENSP00000002165 ...


R[write to console]: Fetching CDS for 1 proteins ... 
R[write to console]: 1 found

R[write to console]: Checking CDS and protein sequence lengths ... 
R[write to console]: 1/1 OK

R[write to console]: Fetching CDS for 1 proteins ... 
R[write to console]: 1 found

R[write to console]: Checking CDS and protein sequence lengths ... 
R[write to console]: 1/1 OK



ENSP00000002596 ...


...

R[write to console]: Fetching CDS for 1 proteins ... 
R[write to console]: 1 found

R[write to console]: Checking CDS and protein sequence lengths ... 
R[write to console]: 1/1 OK

R[write to console]: Fetching CDS for 1 proteins ... 
R[write to console]: 1 found

R[write to console]: Checking CDS and protein sequence lengths ... 
R[write to console]: 1/1 OK



ENSP00000487583 ...


R[write to console]: Fetching CDS for 1 proteins ... 
R[write to console]: 1 found

R[write to console]: Checking CDS and protein sequence lengths ... 
R[write to console]: 1/1 OK

R[write to console]: Fetching CDS for 1 proteins ... 
R[write to console]: 1 found

R[write to console]: Checking CDS and protein sequence lengths ... 
R[write to console]: 1/1 OK



ENSP00000488338 ...


R[write to console]: Fetching CDS for 1 proteins ... 
R[write to console]: 1 found

R[write to console]: Checking CDS and protein sequence lengths ... 
R[write to console]: 1/1 OK

R[write to console]: Fetching CDS for 1 proteins ... 
R[write to console]: 1 found

R[write to console]: Checking CDS and protein sequence lengths ... 
R[write to console]: 1/1 OK



In [9]:
mutsigs = pd.read_csv('human_protein_predictions/MutSig.tsv', sep = "\t", dtype = {'chr': str})

In [10]:
%%R -i mutsigs -o PhyloP

# Use PhyloP                                  
library(GenomicRanges)
library(GenomicScores)
library(IRanges)
library(stringr)

phast <- getGScores("phyloP100way.UCSC.hg38")

PhyloP <- sapply(-1:1, function(x){
  gscores_phylop <- gscores(phast,
        GenomicRanges::GRanges(seqnames = paste0("chr", mutsigs[, "chr"]),
                               ranges = IRanges::IRanges(
                               start = mutsigs[, "g_pos"] + x,
                               end = mutsigs[, "g_pos"] + x)))
  gscores_phylop <- as.data.frame(gscores_phylop)
  return( gscores_phylop[, 6] )
})

colnames( PhyloP ) <- paste("PhyloP", -1:1, sep = "_")
PhyloP <- as.data.frame(PhyloP)

In [11]:
mutsigs = pd.concat([mutsigs.reset_index(drop=True), PhyloP.reset_index(drop=True)], axis = 1)
mutsigs['AA_pos'] = mutsigs['AA_pos'].astype(str)
mutsigs.to_csv('human_protein_predictions/MutSig_PhyloP.tsv', sep="\t", index=False)


In [12]:
mutsigs = pd.read_csv('human_protein_predictions/MutSig_PhyloP.tsv', sep="\t")

/Users/josefng/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3418: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
mutsigs

,protein_id,uniprot_id,chr,g_pos,AA_pos,WT_AA,MUT_AA,WT_codon,MUT_codon,MutSig,PhyloP_-1,PhyloP_0,PhyloP_1
0,ENSP00000001146,Q9NR63,2,72147633.0,1,M,I,ATG,ATA,G[C>T]A,8.0,10.0,-2.0
1,ENSP00000001146,Q9NR63,2,72147631.0,1,M,L,ATG,CTG,A[T>G]G,6.0,8.0,8.0
2,ENSP00000001146,Q9NR63,2,72147632.0,1,M,R,ATG,AGG,A[T>G]G,8.0,8.0,10.0
3,ENSP00000001146,Q9NR63,2,72147632.0,1,M,T,ATG,ACG,A[T>C]G,8.0,8.0,10.0
4,ENSP00000001146,Q9NR63,2,72147633.0,1,M,I,ATG,ATC,G[C>G]A,8.0,10.0,-2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25149041,ENSP00000488338,P07478,7,142774504.0,247,S,N,AGC,AAC,G[C>T]T,0.0,1.5,1.0
25149042,ENSP00000488338,P07478,7,142774503.0,247,S,G,AGC,GGC,C[T>C]G,1.0,0.0,1.5
25149043,ENSP00000488338,P07478,7,142774505.0,247,S,R,AGC,AGG,G[C>G]T,1.5,1.0,2.0
25149044,ENSP00000488338,P07478,7,142774505.0,247,S,R,AGC,AGA,G[C>A]T,1.5,1.0,2.0


In [14]:
# the zoomvar Q(SASA) information
zoomvar = pd.read_csv('human_protein_predictions/ZoomVar.tsv', sep = "\t", index_col=False)
zoomvar = zoomvar[['protein', 'position', 'Q(SASA)']]
zoomvar['position'] = zoomvar['position'].astype(str)
zoomvar

,protein,position,Q(SASA)
0,Q9BRK0,1,None
1,Q9BRK0,2,None
2,Q9BRK0,3,None
3,Q9BRK0,4,None
4,Q9BRK0,5,None
...,...,...,...
4242284,P81274,680,None
4242285,P81274,681,None
4242286,P81274,682,None
4242287,P81274,683,None


In [15]:
%%R -i EVmutation -o EVmutation_collapsed

# keep only 'prediction_epistatic' as the metric to be predicted
# some positions are duplicated (probably because in multiple Pfam defined domains?)
# collapse them by mean(prediction_epistatic)

dups <- which(duplicated(EVmutation[, c(1, 3)]))
dups <- as.numeric( rownames( dups ) )
uni <- EVmutation[ which(! 1:nrow(EVmutation) %in% dups ), ]
dups <- EVmutation[ dups, ]

uni <- uni[, c('uniprot', 'mutant', 'pos', 'wt', 'subs', 'prediction_epistatic' )]

library(plyr)
dups = ddply( EVmutation, c('uniprot', 'mutant', 'pos', 'wt', 'subs'),
              summarise, mean_EVmutation_epistatic = mean( prediction_epistatic, na.rm = TRUE ))
colnames( dups ) <- c('uniprot', 'mutant', 'pos', 'wt', 'subs', 'prediction_epistatic' )

write.table( uni, 'human_protein_predictions/EVmutation_human_protein_predictions_unique.txt', sep = '\t', 
             col.names = TRUE, row.names = FALSE )
write.table( dups, 'human_protein_predictions/EVmutation_human_protein_predictions_dups.txt', sep = '\t', 
             col.names = TRUE, row.names = FALSE )

write.table( rbind( uni, dups ), 'human_protein_predictions/EVmutation_human_protein_predictions_collapsed.txt', 
             sep = '\t', col.names = TRUE, row.names = FALSE )

/home/josefng/.local/lib/python3.8/site-packages/rpy2/robjects/pandas2ri.py:59: UserWarning: Error while trying to convert the column "subs". Fall back to string conversion. The error is: module 'pandas' has no attribute 'NA'
  warnings.warn('Error while trying to convert '


In [16]:
EVmutation = pd.read_csv('human_protein_predictions/EVmutation_human_protein_predictions_collapsed.txt', sep = '\t')

In [17]:
EVmutation

,uniprot,mutant,pos,wt,subs,prediction_epistatic
0,A0AV02,L38A,38,L,A,-6.7742
1,A0AV02,L38C,38,L,C,-7.7904
2,A0AV02,L38D,38,L,D,-6.3475
3,A0AV02,L38E,38,L,E,-6.4752
4,A0AV02,L38F,38,L,F,-7.7904
...,...,...,...,...,...,...
32028175,Q9Y6R7,Y751S,751,Y,S,-9.5550
32028176,Q9Y6R7,Y751T,751,Y,T,-9.5550
32028177,Q9Y6R7,Y751V,751,Y,V,-9.5550
32028178,Q9Y6R7,Y751W,751,Y,W,-6.8902


In [18]:
# merge the mutsigs and zoomvar annotation to the EVmutation table
EVmutation['pos'] = EVmutation['pos'].astype(str)
mutsigs['AA_pos'] = mutsigs['AA_pos'].astype(str)
zoomvar['position'] = zoomvar['position'].astype(str)
EVmutation = pd.merge( EVmutation, mutsigs, \
                       left_on = ["uniprot", "pos", "wt", "subs"], \
                       right_on = ["uniprot_id", "AA_pos", "WT_AA", "MUT_AA"])

EVmutation = pd.merge( EVmutation, zoomvar, left_on = ['uniprot', 'pos'], \
                       right_on = ['protein', 'position'])
EVmutation

,uniprot,mutant,pos,wt,subs,prediction_epistatic,protein_id,uniprot_id,chr,g_pos,...,MUT_AA,WT_codon,MUT_codon,MutSig,PhyloP_-1,PhyloP_0,PhyloP_1,protein,position,Q(SASA)
0,A0AV02,L38M,38,L,M,-7.0228,ENSP00000418783,A0AV02,3,125190435.0,...,M,CTG,ATG,G[C>A]T,8.0,5.0,9.0,A0AV02,38,None
1,A0AV02,L38P,38,L,P,-7.7904,ENSP00000418783,A0AV02,3,125190436.0,...,P,CTG,CCG,C[T>C]G,5.0,9.0,9.0,A0AV02,38,None
2,A0AV02,L38Q,38,L,Q,-3.8096,ENSP00000418783,A0AV02,3,125190436.0,...,Q,CTG,CAG,C[T>A]G,5.0,9.0,9.0,A0AV02,38,None
3,A0AV02,L38R,38,L,R,-3.7020,ENSP00000418783,A0AV02,3,125190436.0,...,R,CTG,CGG,C[T>G]G,5.0,9.0,9.0,A0AV02,38,None
4,A0AV02,L38V,38,L,V,-7.7904,ENSP00000418783,A0AV02,3,125190435.0,...,V,CTG,GTG,G[C>G]T,8.0,5.0,9.0,A0AV02,38,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8249249,Q9Y6Y8,A704G,704,A,G,-4.2731,ENSP00000358071,Q9Y6Y8,10,119920974.0,...,G,GCA,GGA,G[C>G]A,4.0,4.0,0.0,Q9Y6Y8,704,0.4077
8249250,Q9Y6Y8,A704P,704,A,P,-5.3762,ENSP00000358071,Q9Y6Y8,10,119920973.0,...,P,GCA,CCA,G[C>G]T,3.0,4.0,4.0,Q9Y6Y8,704,0.4077
8249251,Q9Y6Y8,A704S,704,A,S,-3.0319,ENSP00000358071,Q9Y6Y8,10,119920973.0,...,S,GCA,TCA,G[C>A]T,3.0,4.0,4.0,Q9Y6Y8,704,0.4077
8249252,Q9Y6Y8,A704T,704,A,T,-4.2223,ENSP00000358071,Q9Y6Y8,10,119920973.0,...,T,GCA,ACA,G[C>T]T,3.0,4.0,4.0,Q9Y6Y8,704,0.4077


In [19]:
# remove those with QSASA nan
EVmutation = EVmutation[ EVmutation['Q(SASA)'].str.find('None') == -1 ]
EVmutation.to_csv('human_protein_predictions/EVmutation_human_protein_predictions_collapsed_annotated.txt', \
                  sep = "\t", index=False)

In [20]:
%%bash

# generate a new file with only BRCA (uniprot P38398) and PTEN (P60484) annotations
head -1 EVmutation_human_protein_predictions_collapsed_annotated.txt > EVmutation_human_protein_predictions_collapsed_annotated_BRCA1_PTEN.txt
grep P38398 EVmutation_human_protein_predictions_collapsed_annotated.txt >> EVmutation_human_protein_predictions_collapsed_annotated_BRCA1_PTEN.txt
grep P60484 EVmutation_human_protein_predictions_collapsed_annotated.txt >> EVmutation_human_protein_predictions_collapsed_annotated_BRCA1_PTEN.txt

# and conversely a file without BRCA and PTEN variants
grep -v -E "P38398|P60484" EVmutation_human_protein_predictions_collapsed_annotated.txt > EVmutation_human_protein_predictions_collapsed_annotated_excludedBRCA1-PTEN.txt

In [21]:
# add DMS exp data onto the BRCA1 and PTEN tables - these are the true labels to be predicted

BRCA1_PTEN = pd.read_csv('human_protein_predictions/EVmutation_human_protein_predictions_collapsed_annotated_BRCA1_PTEN.txt', sep = '\t')
BRCA1_PTEN

,uniprot,mutant,pos,wt,subs,prediction_epistatic,protein_id,uniprot_id,chr,g_pos,...,MUT_AA,WT_codon,MUT_codon,MutSig,PhyloP_-1,PhyloP_0,PhyloP_1,protein,position,Q(SASA)
0,P38398,I21F,21,I,F,-2.5595,ENSP00000350283,P38398,17,43124077.0,...,F,ATC,TTC,A[T>A]T,-3.0,-3.0,1.5,P38398,21,0.3845
1,P38398,I21L,21,I,L,-0.9572,ENSP00000350283,P38398,17,43124077.0,...,L,ATC,CTC,A[T>G]T,-3.0,-3.0,1.5,P38398,21,0.3845
2,P38398,I21M,21,I,M,-0.5942,ENSP00000350283,P38398,17,43124079.0,...,M,ATC,ATG,T[C>G]T,1.5,2.0,1.0,P38398,21,0.3845
3,P38398,I21N,21,I,N,1.5518,ENSP00000350283,P38398,17,43124078.0,...,N,ATC,AAC,A[T>A]C,-3.0,1.5,2.0,P38398,21,0.3845
4,P38398,I21S,21,I,S,-0.1458,ENSP00000350283,P38398,17,43124078.0,...,S,ATC,AGC,A[T>G]C,-3.0,1.5,2.0,P38398,21,0.3845
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1203,P60484,T160I,160,T,I,-6.8576,ENSP00000361021,P60484,10,87933238.0,...,I,ACC,ATC,A[C>T]C,9.0,8.0,4.0,P60484,160,0.1245
1204,P60484,T160N,160,T,N,-7.9631,ENSP00000361021,P60484,10,87933238.0,...,N,ACC,AAC,A[C>A]C,9.0,8.0,4.0,P60484,160,0.1245
1205,P60484,T160P,160,T,P,-7.3228,ENSP00000361021,P60484,10,87933237.0,...,P,ACC,CCC,G[T>G]C,10.0,9.0,8.0,P60484,160,0.1245
1206,P60484,T160S,160,T,S,-4.4731,ENSP00000361021,P60484,10,87933238.0,...,S,ACC,AGC,A[C>G]C,9.0,8.0,4.0,P60484,160,0.1245


In [22]:
BRCA1 = BRCA1_PTEN[BRCA1_PTEN['uniprot'] == 'P38398']
PTEN = BRCA1_PTEN[BRCA1_PTEN['uniprot'] == 'P60484']
PTEN['AA_pos'] = [str(i) for i in PTEN['AA_pos']]

/home/josephn/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
# BRCA1 and PTEN DMS data

# (1) BRCA DMS
BRCA_DMS = pd.read_csv('human_protein_predictions/Findlay_et_al_BRCA_DMS.txt', sep = '\t', skiprows = 2)
BRCA_DMS = BRCA_DMS[BRCA_DMS['func.class'] != 'None']

In [24]:
# function to clean up hgvs-style variant annotation and perform k-means clustering 
# to convert scores into binary labels

from sklearn.cluster import KMeans

def parseAAmut(mut_string, offset):
    """
    Given standard HGVS mutation notation (e.g. 'p.Leu308Pro'), give tuple of
    (WT_AA, MUT_AA, AA_pos) where WT_AA and MUT_AA are One-letter AA codes and AA_pos 
    is an integer (amino acid position of the mutation)
    
    Input:
      mut_string: standard HGVS mutation notation
      offset: int, number of amino acids to add to the AA pos to adjust it to uniprot numbering
    Output:
      tuple of (WT_AA, MUT_AA, AA_pos). semicolon-delimited if multiple AA positions are affected.
    """
    d = {'CYS': 'C', 'ASP': 'D', 'SER': 'S', 'GLN': 'Q', 'LYS': 'K',
     'ILE': 'I', 'PRO': 'P', 'THR': 'T', 'PHE': 'F', 'ASN': 'N', 
     'GLY': 'G', 'HIS': 'H', 'LEU': 'L', 'ARG': 'R', 'TRP': 'W', 
     'ALA': 'A', 'VAL':'V', 'GLU': 'E', 'TYR': 'Y', 'MET': 'M', 'TER': 'X'}
    # if contain frameshift or variants that can't be mapped to AA sequence consequence, ignore this
    if re.search('fs', mut_string) is not None:
        return (None, None, None)
    if mut_string.find('?') != -1:
        return (None, None, None)
    if mut_string.find('*') != -1 and mut_string.find('del') != -1 and mut_string.find('ins') != -1:
        return (None, None, None)
    cmpd = re.search('(?<=p.)\[.*\]', mut_string)
    if cmpd is not None: # compound AA change in the form e.g. p.[Tyr19Cys;Asn22Asp;=]
        cmpd = str(cmpd.group(0)).strip('[]')
        cmpd = cmpd.split(';')
    else:
        cmpd = [mut_string]
    # remove synonymous
    cmpd = [i for i in cmpd if i not in ['p.=', '_wt', 'p.?', '_sy', '=']]
    cmpd = [i for i in cmpd if i.find('?') == -1 and i.find('*') == -1 and i.find('del') == -1 and i.find('ins') == -1]
    if len(cmpd) == 0:
        return (None, None, None)
    AA_pos = list()
    WT_AA = list()
    MUT_AA = list()    
    for ms in cmpd:
        try:
            aa_pos = int(re.search('[0-9]+', ms).group(0))
        except AttributeError:
            print(mut_string)
        try:
            wt = str(re.search('(?<=p.)[A-Za-z]+', ms).group(0)).upper()
        except AttributeError:
            wt = str(re.search('^[A-Za-z]+', ms).group(0)).upper()
        try:
            mut = str(re.search('[A-Za-z\=]+\Z', ms).group(0)).upper()
        except AttributeError:
            print(mut_string)    
        if mut not in d.keys() or wt not in d.keys():
            if mut == '=':
                mut = wt
            else:
                print(ms)
        AA_pos.append( str(aa_pos + offset) )
        WT_AA.append( d[wt] )
        MUT_AA.append( d[mut] )
    return (';'.join(WT_AA), ';'.join(MUT_AA), ';'.join(AA_pos))


def doKMeans2(data):
    """
    Given 'data', perform K-means clustering to yield 2 clusters.
    Label the cluster with higher mean value as '0' and the one with lower mean value is '1'
    Input:
        - data: pd.Series ie one column in the pd DataFrame
    Return:
        np.array of labels in [0, 1]
    """
    data = np.array(data).reshape(-1, 1)
    kmeans = KMeans(n_clusters=2, random_state=123).fit(data)
    labels = kmeans.labels_
    # check mean value
    means = [np.mean([val for i, val in enumerate(data) if labels[i] == j]) for j in [0, 1]]
    if means[0] < means[1]:
        # swap labels
        labels2 = []
        for i, k in enumerate(labels):
            if k == 0:
                labels2.append(1)
            elif k == 1:
                labels2.append(0)
        labels = labels2
    return labels   


In [25]:
# (2) PTEN activity DMS [MAVEdb urn:mavedb:00000054-a-1]
PTEN_act_DMS = pd.read_csv('../DMSexp/data/MAVEdb/urn_mavedb_00000054-a-1.csv', skiprows = 4)
PTEN_act_DMS = PTEN_act_DMS[['hgvs_pro', 'score']]
AAchange = [ parseAAmut(str(mut), 0) for mut in list(PTEN_act_DMS['hgvs_pro']) ] # process `hgvs_pro` column with parseAAmut
PTEN_act_DMS[ ['WT_AA', 'MUT_AA', 'AA_pos'] ] = pd.DataFrame(AAchange, index = PTEN_act_DMS.index)
PTEN_act_DMS = PTEN_act_DMS[ ~ PTEN_act_DMS['score'].isna() ]
kmeans_labels = doKMeans2(PTEN_act_DMS['score'])
PTEN_act_DMS['label'] = pd.DataFrame(kmeans_labels, index = PTEN_act_DMS.index)

PTEN_act_DMS

,hgvs_pro,score,WT_AA,MUT_AA,AA_pos,label
2,p.Val403Glu,0.193065,V,E,403,0
8,p.Val403Gly,-2.736793,V,G,403,1
14,p.Val403Trp,-2.459162,V,W,403,1
17,p.Val403Met,0.106511,V,M,403,0
19,p.Val403Ile,-2.049575,V,I,403,1
...,...,...,...,...,...,...
8673,p.Met1Trp,-1.300060,M,W,1,0
8674,p.Met1Tyr,-0.872034,M,Y,1,0
8676,_wt,1.000000,None,None,None,0
8677,p.Met1Leu,-1.545614,M,L,1,0


In [26]:
# (2) PTEN stability DMS [MAVEdb urn:mavedb:00000013-a-1]
PTEN_stb_DMS = pd.read_csv('../DMSexp/data/MAVEdb/urn_mavedb_00000013-a-1.csv', skiprows = 4)
PTEN_stb_DMS = PTEN_stb_DMS[['hgvs_pro', 'score']]
AAchange = [ parseAAmut(str(mut), 0) for mut in list(PTEN_stb_DMS['hgvs_pro']) ] # process `hgvs_pro` column with parseAAmut
PTEN_stb_DMS[ ['WT_AA', 'MUT_AA', 'AA_pos'] ] = pd.DataFrame(AAchange, index = PTEN_stb_DMS.index)
PTEN_stb_DMS = PTEN_stb_DMS[ ~ PTEN_stb_DMS['score'].isna() ]
kmeans_labels = doKMeans2(PTEN_stb_DMS['score'])
PTEN_stb_DMS['label'] = pd.DataFrame(kmeans_labels, index = PTEN_stb_DMS.index)

PTEN_stb_DMS

,hgvs_pro,score,WT_AA,MUT_AA,AA_pos,label
0,p.Met1Val,1.065143,M,V,1,0
1,p.Thr2Ser,0.824359,T,S,2,0
2,p.Asn12Ile,0.298717,N,I,12,1
3,p.Arg74Val,0.855027,R,V,74,0
4,p.His75Ala,0.863776,H,A,75,0
...,...,...,...,...,...,...
4404,p.Arg74Asn,1.158127,R,N,74,0
4405,p.Arg74Pro,1.059395,R,P,74,0
4406,p.Arg74=,1.156741,R,R,74,0
4407,p.Arg74Ser,0.868741,R,S,74,0


In [27]:
# merge these to the annotated EVmutation data - these scores/labels are used as the 'true'
# labels to validate the predictions

BRCA_DMS = pd.merge( BRCA_DMS, BRCA1, \
                     left_on = ['aa_pos', 'aa_ref', 'aa_alt'], \
                     right_on = ['AA_pos', 'WT_AA', 'MUT_AA'])
PTEN_act_DMS = pd.merge( PTEN_act_DMS, PTEN, \
                         left_on = ['AA_pos', 'WT_AA', 'MUT_AA'], \
                         right_on = ['AA_pos', 'WT_AA', 'MUT_AA'])
PTEN_stb_DMS = pd.merge( PTEN_stb_DMS, PTEN, \
                         left_on = ['AA_pos', 'WT_AA', 'MUT_AA'], \
                         right_on = ['AA_pos', 'WT_AA', 'MUT_AA'])


In [28]:
BRCA_DMS.to_csv('human_protein_predictions/BRCA_DMS_annotated.txt', sep = '\t', index = False)
PTEN_act_DMS.to_csv('human_protein_predictions/PTEN_activity_DMS_annotated.txt', sep = '\t', index = False)
PTEN_stb_DMS.to_csv('human_protein_predictions/PTEN_stability_DMS_annotated.txt', sep = '\t', index = False)

## prepare data

The following preparation / normalisation:

1. parse DNA substitution type & base at position -1 and +1 separately into three features
2. Normalise PhyloP score so they are in range [-1, 1]
3. Scale Q(SASA) by subtracting 0.15. So that +ve value means surface (exposed) and -ve value means core (buried)
4. One-hot encoding of the non-numeric features.

In [29]:
# functions for one-hot encoding

def decomposeMutSig(mutsig):
    """
    given a mutsig string of format 'W[X>Y]Z', decompose into position -1, +1 and the actual substitution
    Input:
        mutsig: string of format 'W[X>Y]Z'
    Output:
        tuple of (pos-1, pos+1, mut). So 'W[X>Y]Z' --> ('W', 'Z', 'X>Y')
    """
    pos_minus1 = mutsig[0]
    pos_plus1 = mutsig[-1]
    mut = re.search('[AGCT]>[AGCT]', mutsig).group(0)
    return (pos_minus1, pos_plus1, mut)
    

def onehot(data, levels):
    """
    one-hot encoding of the 'data'.
    Input:
        data - the list/pd.Series of data to be encoded
        levels - list of possible 'factor' levels of this list/series
    Output:
        np.array of shape (n_data, n_levels)
    """
    encode = np.zeros((len(data), len(levels)))
    for i, val in enumerate(data):
        encode[i, levels.index(val)] = 1
    return encode    

def prepare_data(file, delimiter, output_col, WT_AA = True, MUT_AA = True, \
                Pos_1 = True, Subs = True, PhyloP = True, QSASA = True):
    """
    wrapper function to read in the data and do required preprocessing
    Input:
        file - str, filepath. Has header in the first column and no row index columns.
        delimiter - str, delimiter used in 'file'
        output_col, str, column name in `file` corresponding to the output labels 'Y' 
    Output: 
        tuple of 2 or 3 elements:
        - np.array of shape (db.shape[0], 58) where db is the resulting data frame from reading in `file`. These are the features 'X'.
        - np.array of shape (db.shape[0], 1). These are the output 'Y' to be predicted.
    """
    db = pd.read_csv(file, sep = delimiter, header=0, index_col=False, na_values = 'None')
    print('Data read in. Now process them')
    if PhyloP:
        # normalisation: PhyloP score columns / 10 so that it ranges from -1 to 1.
        db[['PhyloP_-1', 'PhyloP_0', 'PhyloP_1']] /= 10
        print('PhyloP score normalised to [-1, 1].')

    # decomposeMutSig on 'MutSig'
    db[['Pos_-1', 'Pos_+1', 'Mut']] = pd.DataFrame([decomposeMutSig(val) for val in db['MutSig']], index = db.index)
    print('MutSig parsed.')

    # one-hot on the new columns, WT_AA, MUT_AA.
    AAs = list('ACDEFGHIKLMNPQRSTVWY')
    feature_cols = []
    if WT_AA:
        print('WT_AA one-hot encoding ...')
        db[ ['WT_AA_' + aa for aa in AAs] ] = pd.DataFrame(onehot(db['WT_AA'], AAs), index = db.index)
        feature_cols +=  ['WT_AA_' + aa for aa in AAs]
    if MUT_AA:
        print('MUT_AA one-hot encoding ...')
        db[ ['MUT_AA_' + aa for aa in AAs] ] = pd.DataFrame(onehot(db['MUT_AA'], AAs), index = db.index)
        feature_cols +=  ['MUT_AA_' + aa for aa in AAs]
    bs = list('AGCT')
    if Pos_1:
        print('-1 and +1 DNA pos one-hot encoding ...')
        db[ ['Pos_-1_' + b for b in bs] ] = pd.DataFrame(onehot(db['Pos_-1'], bs), index = db.index)
        db[ ['Pos_+1_' + b for b in bs] ] = pd.DataFrame(onehot(db['Pos_+1'], bs), index = db.index)
        feature_cols +=  ['Pos_-1_' + b for b in bs]
        feature_cols +=  ['Pos_+1_' + b for b in bs]
    subs = ['C>A', 'C>G', 'C>T', 'T>A', 'T>C', 'T>G']
    if Subs:    
        print('DNA substitution one-hot encoding ...')
        db[ ['Mut_' + m for m in subs] ] = pd.DataFrame(onehot(db['Mut'], subs), index = db.index)
        feature_cols +=  ['Mut_' + m for m in subs] 
    if PhyloP:
        feature_cols += ['PhyloP_0']
        if Pos_1:
            feature_cols +=  ['PhyloP_-1', 'PhyloP_1']
    if QSASA:
        db['Q(SASA)'] -= 0.15
        feature_cols += ['Q(SASA)']
    if 'scoreset' in db.columns:
        return np.array( db[ feature_cols] ), np.array( db[output_col] ).reshape((-1,1)), np.array(db['scoreset']).reshape((-1, 1))
    else:
        return np.array( db[ feature_cols] ), np.array( db[output_col] ).reshape((-1,1))

In [30]:
X, Y = prepare_data('human_protein_predictions/EVmutation_human_protein_predictions_collapsed_annotated.txt', \
                    delimiter="\t", output_col = "prediction_epistatic")

/Users/josefng/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  


Data read in. Now process them
PhyloP score normalised to [-1, 1].
MutSig parsed.
WT_AA one-hot encoding ...
MUT_AA one-hot encoding ...
-1 and +1 DNA pos one-hot encoding ...
DNA substitution one-hot encoding ...


In [31]:
print(X.shape, Y.shape)

(5805302, 58) (5805302, 1)


~6M variants for predictor.

In [32]:
# save the objects
pickle.dump( (X, Y), open('human_protein_predictions/EVmutation_human_protein_predictions_collapsed_annotated.pickle', 'wb') )

In [33]:
# EVmutation data but hold-out BRCA and PTEN
X, Y = prepare_data('human_protein_predictions/EVmutation_human_protein_predictions_collapsed_annotated_excludedBRCA1-PTEN.txt', \
                    delimiter="\t", output_col = "prediction_epistatic")
pickle.dump( (X, Y), open('human_protein_predictions/EVmutation_human_protein_predictions_collapsed_annotated_excludedBRCA1-PTEN.pickle', 'wb') )

/home/josephn/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3263: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Data read in. Now process them
PhyloP score normalised to [-1, 1].
MutSig parsed.
WT_AA one-hot encoding ...
MUT_AA one-hot encoding ...
-1 and +1 DNA pos one-hot encoding ...
DNA substitution one-hot encoding ...


In [34]:
# BRCA DMS
X, Y = prepare_data('human_protein_predictions/BRCA_DMS_annotated.txt', \
                    delimiter="\t", output_col = "func.class")
pickle.dump( (X, Y), open('human_protein_predictions/BRCA_DMS_annotated.pickle', 'wb') )

Data read in. Now process them
PhyloP score normalised to [-1, 1].
MutSig parsed.
WT_AA one-hot encoding ...
MUT_AA one-hot encoding ...
-1 and +1 DNA pos one-hot encoding ...
DNA substitution one-hot encoding ...


In [35]:
# PTEN activity DMS
X, Y = prepare_data('human_protein_predictions/PTEN_activity_DMS_annotated.txt', \
                    delimiter="\t", output_col = "label")
pickle.dump( (X, Y), open('human_protein_predictions/PTEN_activity_DMS_annotated.pickle', 'wb') )

Data read in. Now process them
PhyloP score normalised to [-1, 1].
MutSig parsed.
WT_AA one-hot encoding ...
MUT_AA one-hot encoding ...
-1 and +1 DNA pos one-hot encoding ...
DNA substitution one-hot encoding ...


In [36]:
# PTEN stability DMS
X, Y = prepare_data('human_protein_predictions/PTEN_stability_DMS_annotated.txt', \
                    delimiter="\t", output_col = "label")
pickle.dump( (X, Y), open('human_protein_predictions/PTEN_stability_DMS_annotated.pickle', 'wb') )

Data read in. Now process them
PhyloP score normalised to [-1, 1].
MutSig parsed.
WT_AA one-hot encoding ...
MUT_AA one-hot encoding ...
-1 and +1 DNA pos one-hot encoding ...
DNA substitution one-hot encoding ...
